In [176]:
import random
import numpy as np
import pandas as pd

# Music Practice Builder
- Load data
- create empty practice session
- split into essential and non-essential items
- add essential items to session
- sort by priority


## Configuration

In [177]:
input_file = './practice_elements.xlsx'
practice_time_minutes = 45

## Slicing and dicing the data

In [178]:
data = pd.read_excel(
    input_file, 
    header=0,
    sheet='items',
    converters=
    {
        'min_time': int,
        'max_time': int,
        'priority': float,
        'essential': bool,
    })

In [179]:
session = data.query('essential == True')

In [180]:
items = data.query('essential == False')

## Generate the initial set of random times for the essential items

In [181]:
times_df = pd.DataFrame({'time':session.apply(lambda row: random.randrange(row.min_time, row.max_time+1), axis=1)}, 
                        index=session.index)
session = session.join(times_df)

## Fill the rest of the session

In [182]:
num_attempts_to_fill_time = 0
max_attempts_to_fill_time = 100
while session.time.sum() < practice_time_minutes and len(items) > 0:
    i = items.sample(n=1, weights='weight')
    items = items.drop(i.index)
    i['time'] = random.randrange(i.min_time.iloc[0], i.max_time.iloc[0]+1)
    if session.time.sum() + i.time.sum() <= practice_time_minutes:
        session = session.append(i)
    else:
        num_attempts_to_fill_time += 1
        if num_attempts_to_fill_time < max_attempts_to_fill_time:
            print(num_attempts_to_fill_time)
            continue
        else:
            print('Bailing out, may not have filled all time')
            break

1
2
3
4
5
6


# Today's Practice Session

In [183]:
print('Target time: {0}, Actual time: {1}'.format(practice_time_minutes, session.time.sum()))

Target time: 45, Actual time: 44


In [184]:
session.sort_values(by='sort_order')[['name', 'category', 'time']]

,name,category,time
5,must do first,technique,3
6,t2,technique,3
12,s2,sight reading,5
2,p3,pattern,3
11,s1,sight reading,5
4,p5,pattern,3
0,p1,pattern,4
1,p2,pattern,3
10,r1,repertoire,10
15,r2,repertoire,5
